# Create SHETRAN Raster Data
*Ben Smith | 12/12/2025*

This script is designed to take online downloads and reconfigure them into raster layers that can be used to setup SHETRAN models.

Todo:
- Run at 100m, 200m, 500m and 1000m.
- Consider the fixes for the catchments that are below sea level (but that may be one for a later script).
- TODO: Change Satmarsh from bareground to Shrub and recreate Land Use map.

### Preamble

In [103]:
import os
import zipfile

import rasterio
import numpy as np

# from scipy.ndimage import generic_filter
# import geopandas as gpd
import pandas as pd

# from rasterio.merge import merge
# import rasterio.features
# from shapely.geometry import box
# import math
# from rasterio import open as rio_open
# from rasterio.warp import calculate_default_transform, reproject, Resampling
# from rasterio.transform import from_bounds
from rasterio.features import rasterize

import geopandas as gpd

root = 'I:/SHETRAN_GB_2021/02_Input_Data/01 - National Data Inputs for SHETRAN UK/'
resolution_output = 1000

def write_ascii(
        array: np,
        ascii_ouput_path: str,
        xllcorner: float,
        yllcorner: float,
        cellsize: float,
        ncols: int = None,
        nrows: int = None,
        NODATA_value: int = -9999,
        data_format: str = '%1.1f'):

        if len(array.shape) > 0:
            nrows, ncols = array.shape

        file_head = "\n".join(
            ["ncols         " + str(ncols),
             "nrows         " + str(nrows),
             "xllcorner     " + str(xllcorner),
             "yllcorner     " + str(yllcorner),
             "cellsize      " + str(cellsize),
             "NODATA_value  " + str(NODATA_value)])

        with open(ascii_ouput_path, 'wb') as output_filepath:
            np.savetxt(fname=output_filepath, X=array,
                       delimiter=' ', newline='\n', fmt=data_format, comments="",
                       header=file_head
                       )


def read_ascii_raster(file_path, data_type=int, return_metadata=True, replace_NA=False):
    """
    Read ascii raster into numpy array, optionally returning headers.
    """
    headers = []
    dc = {}
    with open(file_path, 'r') as fh:
        for i in range(6):
            asc_line = fh.readline()
            headers.append(asc_line.rstrip())
            key, val = asc_line.rstrip().split()
            dc[key] = val
    ncols = int(dc['ncols'])
    nrows = int(dc['nrows'])
    xll = float(dc['xllcorner'])
    yll = float(dc['yllcorner'])
    cellsize = float(dc['cellsize'])
    nodata = float(dc['NODATA_value'])

    arr = np.loadtxt(file_path, dtype=data_type, skiprows=6)
    if replace_NA:
       arr[arr==nodata] = np.nan

    headers = '\n'.join(headers)
    headers = headers.rstrip()

    if return_metadata:
        return arr, ncols, nrows, xll, yll, cellsize, nodata, headers, dc
    else:
        return arr

# Function for cell aggregation
def cell_reduce(array, block_size, func=np.nanmean):
    """
    Resample a NumPy array by reducing its resolution using block aggregation.
    Parameters:
    - array: Input NumPy array.
    - block_size: Factor by which to reduce the resolution.
    - func: Aggregation function (e.g., np.nanmean, np.nanmin, np.nanmax).
            Recomended to use nanmean etc. else you will lose coverage
    """
    shape = (array.shape[0] // block_size, block_size, array.shape[1] // block_size, block_size,)

    return func(array.reshape(shape), axis=(1, 3), )


# Define a function to calculate the mean of valid neighbors:
def fill_holes(values):
    # This will fill all holes with a value in a neighboring cell.

    center = values[4]  # Center pixel in the 3x3 window
    if np.isnan(center):  # If the center is a hole
        neighbors = values[np.arange(len(values)) != 4]  # Exclude the center
        valid_neighbors = neighbors[~np.isnan(neighbors)]  # Keep valid neighbors
        if len(valid_neighbors) > 0:  # Fill only if there are valid neighbors
            return valid_neighbors.mean()
    return center  # Return the original value if not a hole

# Create a function for simplifying map and table data by removing duplicates:
def remove_map_df_duplicates(map_path, table_path, ID_col, duplicate_columns, output_suffix='_unique', data_format: str = '%1.1f'):
    """
    Function to remove duplicate entries in a raster and a linked dataframe.
    The duplicates are identified based on the columns in duplicate_columns, and the minimum Raster_ID is used for each group.
    ID_col: string of ID column. e.g. 'Raster_ID'
    duplicate_columns = list of column srings. e.g. ['Flow Mechanism', 'Summary']
    """

    # Read in the table and map:
    table = pd.read_csv(table_path)
    map, mc, mr, mx, my, mcs, mnd, _, _ = read_ascii_raster(map_path, data_type=int, replace_NA=False)

    # Group using the desired columns and return the raster IDs in each group:
    groups = table.groupby(duplicate_columns)[ID_col].apply(list).reset_index().Raster_ID

    # -- Now run through each group, finding the minimum Raster_ID and changing all other IDs to that in the map.

    # Run throug the groups:
    for group in groups:
        # Find minimum ID: 
        new_ID = min(group)

        # Change the duplicated IDs to the new ID:
        for old_ID in group:
            ## Table
            # table.loc[table[ID_col] == old_ID, ID_col] = new_ID
            # Map
            map[map == old_ID] = new_ID

    # Drop duplicates from the tbale - the method above will change the IDs but will not remove duplicate rows:
    table.drop_duplicates(subset=duplicate_columns, keep='first', inplace=True)#.reset_index(drop=True)

    # Reset the indexes so that they run consecutively:
    counter = 1
    for old_ID in sorted(table[ID_col]):
        # Table:
        table.loc[table[ID_col] == old_ID, ID_col] = counter
        # Map
        map[map == old_ID] = counter
        counter+=1

    # Write out the map:
    write_ascii(array=map, ascii_ouput_path=map_path.replace('.asc', f'{output_suffix}.asc'),
        xllcorner=mx, yllcorner=my, cellsize=mcs, ncols=mc, nrows=mr, NODATA_value=mnd, data_format=data_format)

    # Remove the duplicated rows from the table and write it to csv:
    table.to_csv(table_path.replace('.csv', f'{output_suffix}.csv'), index=False)


## Elevation Data

Elevation data for the DEM and minDEM is taken from the OS Terrain 50 dataset. This is free to download:
https://osdatahub.os.uk/downloads/open/Terrain50

Around the coastline, the OS data shows the sea using negative values (presumably taken from a low resolution bathymetry map). It is presumed that this will not impact SHETRAN elevations going forward as the setups do not run to the coast. If much larger negative values were used (i.e. -9999) then this may have a greater impact on those coastal cells compared to the current OS values (0 to -2m or so); although these would still be unlikely to be included within the model domains.

This is used to create the DEM and minimum DEM (which is used for rivers).

OSNI 50m data for Northern Ireland was downloaded as a csv of points. These were converted into an ascii grid using QGIS:
 1. Reprojected from ING to BNG.
2. Converted from points to gridded raster with extents rounded to the appropriate 50m.
3. No data cells (where there were no points in a raster cell) were filled using Fill No Data, ensuring to only look 1 cell away for a value. This does fill some water cells that should be missing data, but this is non-consequential.
4. This filling process was repeated a few times to fill in gaps in the dats where there are lakes etc. Again, non-consequential.
5. Data written as an ascii grid for incorporation into the rasters below. You can use QGIS's _Convert Format_ with _Additional command line parameters_ '-co DECIMAL_PRECISION=1' to write this with 1 decimal place to reduce file size.
6. The NI data would not immediately merge with the GB data due to an issue with the projection. These were very similar (see below), and so I simply copied a GB projection from a prj file to the NI prj file... I don't think this makes any tangible difference.

GB Projection:
<code>
PROJCS["British_National_Grid",GEOGCS["GCS_OSGB_1936",DATUM["D_OSGB_1936",SPHEROID["Airy_1830",6377563.396,299.3249646]],PRIMEM["Greenwich",0],UNIT["Degree",0.017453292519943295]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",400000],PARAMETER["False_Northing",-100000],PARAMETER["Central_Meridian",-2],PARAMETER["Scale_Factor",0.999601272],PARAMETER["Latitude_Of_Origin",49],UNIT["Meter",1]]
</code>

Original NI Projection
<code>
PROJCS["British_National_Grid",GEOGCS["GCS_OSGB_1936",DATUM["D_OSGB_1936",SPHEROID["Airy_1830",6377563.396,299.3249646]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",400000.0],PARAMETER["False_Northing",-100000.0],PARAMETER["Central_Meridian",-2.0],PARAMETER["Scale_Factor",0.9996012717],PARAMETER["Latitude_Of_Origin",49.0],UNIT["Meter",1.0]]
</code>



In [2]:
# The data is within sub-folders, list these:
OS50_zip_path = os.path.join(root, "terr50_gagg_gb/data/")
OS50_zip_folders = os.listdir(OS50_zip_path)
OS50_zip_folders = [a for a in OS50_zip_folders if 'Unzipped_data' not in a]

# Setup a new folder to hold the unzipped data:
OS50_unzipped_folder = os.path.join(OS50_zip_path, 'Unzipped_data/')
if not os.path.exists(OS50_unzipped_folder):
    os.mkdir(OS50_unzipped_folder)

# Unzip the data:
for OS50_zip_folder in OS50_zip_folders:
    zip_folders = os.listdir(os.path.join(OS50_zip_path, OS50_zip_folder))
    for zip_folder in zip_folders:
        print(os.path.join(OS50_zip_path, OS50_zip_folder, zip_folder))
        with zipfile.ZipFile(os.path.join(OS50_zip_path, OS50_zip_folder, zip_folder), 'r') as zip_ref:
            zip_ref.extractall(OS50_unzipped_folder)

Join the elevation rasters into a single file.

In [ ]:
# List all .asc files in the folder
asc_files = [os.path.join(OS50_unzipped_folder, f) for f in os.listdir(OS50_unzipped_folder) if f.endswith('.asc')]

# Open the GB files using rasterio:
count = 1
raster_list = []
for asc_file in asc_files:
    print(count, "/", len(asc_files))
    raster = rasterio.open(asc_file,)
    raster_list.append(raster)
    count += 1

# ---

# Open the filled NI file using rasterio:
print('NI', "/", len(asc_files))
raster = rasterio.open(os.path.join(root, 'OSNI_OpenData_50m/OSNI_OpenData_50m_BNG_Filled.asc'),)
raster_list.append(raster)

# ---

# Combine (merge) the rasters:
merged_raster, merged_transform = merge(raster_list, nodata=-9999)

# Close the opened raster files - you may be able to incorporate this into the loop above.
for raster in raster_list:
    raster.close()

# Extract the first raster band and change 0s to -9999:
merged_raster = merged_raster[0]
# merged_raster[merged_raster == 0] = -9999  # This was changed to merge(..., nodata=-9999) as it created issues in the fens

National_OS50_path = os.path.join(root, 'Processed Data', 'National_OS50.asc')

# Write the file as an ascii:
write_ascii(
    array=merged_raster,
    ascii_ouput_path=National_OS50_path,
    xllcorner=merged_transform[2],
    yllcorner=merged_transform[5]-(merged_raster.shape[0]*merged_transform[0]),
    cellsize=merged_transform[0],
)

Regrid the elevation rasters to the desired size.

Note that this does assume that the lower left corner of the national OS50 file is at 0,0, easting northing. Check this if you are redoing this work. you can load the header of the file using the following code:
<code>
headers = []
with open(OS50_zip_path + 'National_OS50.asc', 'r') as fh:
for i in range(6):
asc_line = fh.readline()
headers.append(asc_line.rstrip())
headers
</code>

The first stage of this is to ensure that the 50m data is of the same extent as the 1km data. Rows and columns are added to ensure this. This means that the data has an extent that is in 1km, so can be resampled to divisions of this (1km, 500m, 200m, 100m). This may not work if you try other resolutions as, because the calculations will run from the top left, not the bottom left, the resampled dataset may not have llx/lly coordinates of 0,0. Think about this if you want to use other resolutions!

In [ ]:
national_OS50, _, _, _, _, _, _, _, OS50_header = read_ascii_raster(National_OS50_path, data_type=float, replace_NA=True)

In [ ]:
# # If you have not loaded in the dataset (perhaps because you are only testing the code), you can check the dimensions of the 50m dataset using this code:
#
# OS50_header = {}
# with open(OS50_zip_path + 'National_OS50.asc', 'r') as fh:
#     for i in range(6):
#         asc_line = fh.readline()
#         key, val = asc_line.rstrip().split()
#         OS50_header[key] = val
# OS50_header

In [ ]:
# Resize the national dataset to match existing SHETRAN inputs:
# Resize the inputs to the desired SHETRAN grid (top right corner should be x: 661000, y: 1241000):
row_difference = int(((1241*1000) - float(OS50_header['nrows']) * float(OS50_header['cellsize'])) / float(OS50_header['cellsize']))
col_difference = int(((661*1000) - float(OS50_header['ncols']) * float(OS50_header['cellsize'])) / float(OS50_header['cellsize']))

if row_difference > 0:
    # Create the rows of -9999
    new_rows = np.full((row_difference, national_OS50.shape[1]), -9999)
    # Add the new rows to the top
    national_OS50 = np.vstack((new_rows, national_OS50))

# repeat for columns:
if row_difference > 0:
    new_cols = np.full((national_OS50.shape[0], col_difference), -9999)
    national_OS50 = np.hstack((national_OS50, new_cols))  # Remember that these need adding at the end/right.

_I have removed the code chuck below as I think it is superfluous. There were some issues resulting from changing 0 values to NA when in fact these are valid elevations. This has been corrected and the code designed to fix/fill the holes left below in case of potential future uses.

*_it may be of use in the Northern Ireland catchments, where there is a greater presence of NA values over lakes._*

_This will fill the holes (na/-9999 values) in the dataset - this code will only fill calls that have a valid value in an adjacent cell._

<code>
\# Replace hole_value with NaN for processing
raster[raster == -9999] = np.nan
\# Apply the function iteratively
filled_national_OS50 = generic_filter(national_OS50, fill_holes, size=3, mode='constant', cval=np.nan)
filled_national_OS50[filled_national_OS50 == np.nan] = -9999
\# Write the file as an ascii:
write_ascii(
    array=filled_national_OS50,
    ascii_ouput_path=f'{OS50_zip_path}National_OS50_DEM_preprocessed.asc',
    xllcorner=OS50_header['xllcorner'],
    yllcorner=OS50_header['yllcorner'],
    cellsize=float(OS50_header['cellsize'])
)
</code>

**The following code will give warnings when trying to take the mean of cells that are all np.nan - don't worry, this is doing what it should. (Probably everything in QGIS or similar at the end though).**

In [ ]:
# Define the block size for aggregation
resolution_input = float(OS50_header['cellsize'])
block_size = int(resolution_output/resolution_input)  # For 50m -> 100m, use a block size of 2

# Resample using the mean and minimum:
DEM = cell_reduce(national_OS50, block_size, np.mean)
minDEM = cell_reduce(national_OS50, block_size, np.min)

# -9999 was converted to np.nan in the loading phase, convert it back
DEM[np.isnan(DEM)] = -9999
minDEM[np.isnan(minDEM)] = -9999

In [3]:
# Write the file as an ascii:
write_ascii(
    array=DEM,
    ascii_ouput_path=f'{root}/Processed Data/National_OS50_DEM_{resolution_output}m.asc',
    xllcorner=OS50_header['xllcorner'],
    yllcorner=OS50_header['yllcorner'],
    cellsize=resolution_output
)

# Write the file as an ascii:
write_ascii(
    array=minDEM,
    ascii_ouput_path=f'{root}/Processed Data/National_OS50_minDEM_{resolution_output}m.asc',
    xllcorner=OS50_header['xllcorner'],
    yllcorner=OS50_header['yllcorner'],
    cellsize=resolution_output
)


## Land Cover Datasets

These are available as 25m and 1km rasters or as vector layers. Vectors are prefered as these allow for greater precision when building lower resolution rasters however these took an unfeasibly long time to process and so rasters were used instead. On inspection these look fully fit for purpose.

All data is CEH Land Cover data (2007), available online for GB and NI (separately):
_https://catalogue.ceh.ac.uk/documents/e02b4228-fdcf-4ab7-8d9d-d3a16441e23d_

The NI data was converted from ING to BNG and then merged with the GB data. The CEH Land use classes were then reclassified to the SHETRAN classes and the data resampled to the required resolution and written as a .asc file in the same format as the other SHETRAN inputs.

| 	**LCM2007 Class**	       | 	**LCM2007 Class Number**	 | 	**SHETRAN Class**	 | 	**SHETRAN Class Number**	 |
|---------------------------|----------------------------|---------------------|----------------------------|
| 	Broadleaved woodland	    | 	1	                        | 	DeciduousForest	   | 	4	                        |
| 	Coniferous Woodland	     | 	2	                        | 	EvergreenForest	   | 	5	                        |
| 	Arable and Horticulture	 | 	3	                        | 	Arable	            | 	1	                        |
| 	Improved Grassland	      | 	4	                        | 	Grass	             | 	3	                        |
| 	Rough grassland	         | 	5	                        | 	Grass	             | 	3	                        |
| 	Neutral Grassland	       | 	6	                        | 	Grass	             | 	3	                        |
| 	Calcareous Grassland	    | 	7	                        | 	Grass	             | 	3	                        |
| 	Acid Grassland	          | 	8	                        | 	Grass	             | 	3	                        |
| 	Fen, Marsh and Swamp	    | 	9	                        | 	Shrub	             | 	6	                        |
| 	Heather	                 | 	10	                       | 	Shrub	             | 	6	                        |
| 	Heather grassland	       | 	11	                       | 	Shrub	             | 	6	                        |
| 	Bog	                     | 	12	                       | 	Shrub	             | 	6	                        |
| 	Montane Habitats	        | 	13	                       | 	Shrub	             | 	6	                        |
| 	Inland Rock	             | 	14	                       | 	BareGround	        | 	2	                        |
| 	Saltwater	               | 	15	                       | 	Water	             | 	8	                        |
| 	Freshwater	              | 	16	                       | 	Water	             | 	8	                        |
| 	Supra-littoral Rock	     | 	17	                       | 	BareGround	        | 	2	                        |
| 	Supra-littoral Sediment	 | 	18	                       | 	BareGround	        | 	2	                        |
| 	Littoral Rock	           | 	19	                       | 	BareGround	        | 	2	                        |
| 	Littoral sediment	       | 	20	                       | 	BareGround	        | 	2	                        |
| 	Saltmarsh                | 21	                        | Shrub               | 6 |
| 	Urban	                   | 	22	                       | 	Urban	             | 	7	                        |
| 	Suburban	                | 	23	                       | 	Urban	             | 	7	                        |

**Step 1 - Reproject the NI data**

This reprojection of the NI data from ING to BNG does affect it slightly, but that shouldn't make much difference in the models.

In [27]:
# >>> Reproject NI data to BNG - Run once! <<<
# Define input and output file paths
NI_LCM = root + "/Land Use Inputs/LCM 2007 25m Raster/data/LCM2007_NI_25M.tif"
NI_LCM_BNG = root + "/Land Use Inputs/LCM 2007 25m Raster/data/LCM2007_NI_25M_BNG.tif"

# Reproject raster
with rasterio.open(NI_LCM) as src:
    # Define target CRS (British National Grid)
    dst_crs = "EPSG:27700"

    # Calculate transform and dimensions for the target CRS
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds
    )
    kwargs = src.meta.copy()
    kwargs.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })

    # Write the reprojected raster
    with rasterio.open(NI_LCM_BNG, 'w', **kwargs) as dst:
        reproject(
            source=rasterio.band(src, 1),  # Source data
            destination=rasterio.band(dst, 1),  # Destination array
            src_transform=src.transform,
            src_crs=src.crs,
            dst_transform=transform,
            dst_crs=dst_crs,
            resampling=Resampling.nearest  # Adjust resampling method if needed
        )

**Step 2 - Merge, reclassify, resample to the desired resolution and write**

Change the resolution at the top of this file and rerun for whatever resolutions you need. This seems to be slower for the larger resolutions.

In [21]:
# Reclassification mapping
reclass_mapping = {0: -9999,
    1: 4, 2: 5, 3: 1, 4: 3, 5: 3, 6: 3, 7: 3, 8: 3, 9: 6,
    10: 6, 11: 6, 12: 6, 13: 6, 14: 2, 15: 8, 16: 8,
    17: 2, 18: 2, 19: 2, 20: 2, 21: 2, 22: 7, 23: 7
}

# Paths for your rasters
raster_GB_LCM = root + "/Land Use Inputs/LCM 2007 25m Raster/data/lcm2007gb25m.tif"
raster_NI_LCM = root + "/Land Use Inputs/LCM 2007 25m Raster/data/LCM2007_NI_25M_BNG.tif"

# Open LCM GB and NI raster files:
print('READING')
rasters = [rasterio.open(f) for f in [raster_GB_LCM, raster_NI_LCM]]

# Merge the rasters into a single UK raster:
merged_raster, merged_transform = merge(rasters)

# Create an empty array to hold the reclassified data:
reclassified_data = np.empty(merged_raster.shape)  # np.copy(merged_raster)

# Reclassify from the LCM classes the SHETRAN classes:
for original_value, new_value in reclass_mapping.items():
    reclassified_data[merged_raster == original_value] = new_value

# Change -9999s into nan values so that they do not influence processing:
reclassified_data[reclassified_data == -9999] = np.nan

# Set up an empty array to hole the resampled data:
xmin, ymin, xmax, ymax = 0, 0, 661000, 1241000  # resolution of the existing SHETRAN inputs
new_transform = from_bounds(xmin, ymin, xmax, ymax,
                            width=(xmax - xmin) // resolution_output,
                            height=(ymax - ymin) // resolution_output)
new_shape = ((ymax - ymin) // resolution_output, (xmax - xmin) // resolution_output)
resampled_raster = np.empty(new_shape)

# Resample the data to the desired resolution using the most common land use in each cell (the modal class):
reproject(  # You could also do this by applying the row_difference and cell_reduce method from the DEM.
    source=reclassified_data, destination=resampled_raster, src_transform=merged_transform,
    src_crs="EPSG:27700", dst_transform=new_transform, dst_crs="EPSG:27700",
    resampling=Resampling.mode  # Use the mode to get the value that is most common
)

# Change np.nan's back into -9999s:
resampled_raster[np.isnan(resampled_raster)] = -9999

# Write as an asc file:
output_path = f'{root}/Processed Data/UK Land Use {resolution_output}m'

# Save to file
with rasterio.open(
        output_path+'.asc', "w", driver="AAIGrid", height=resampled_raster.shape[0], width=resampled_raster.shape[1],
        count=1, dtype=resampled_raster.dtype, crs="EPSG:27700", transform=new_transform, nodata=-9999) as dst:
    dst.write(resampled_raster, 1)

## Lake Map

Use the soil data grid to build a raster that masks out the lakes. In previous versions, OS data was used to define lakes; however, when reviewing this it appears that this method is over zealous.

The lake map is used to alter the river parameters  in cells that contain lakes. Any river channel that is next to a lake cell have the Strickler overland flow reduced from either 20 to 3 or 50 to 10 (depending on which SHETRAN version is being used).

This is created using the CEH land use data but with a lower threshold for lake presence than the land use raster created above. This is because using the *mode* to describe the presence or absence of a lake tends to underestimate. Instead, a numpy array of 0's is built, water cells are added to it as 1's, then in the regridding process the average of the input cells is taken. This can then be compared to a threshold and cells >= the threshold set to lakes (1) and all other cells set to -9999.

A threshold of around 25% creates a lake map that (at 1km) matches relatively well to reality.

*TODO: Consider cross referencing this with OS lake data to remove rivers from the lakes dataset.*

In [ ]:
# Create an empty array to hold the reclassified data:
reclassified_lake_data = np.zeros(shape=merged_raster.shape)  # np.copy(merged_raster)

# Change -9999s into nan values so that they do not influence processing:
reclassified_lake_data[merged_raster == 15] = 1
reclassified_lake_data[merged_raster == 16] = 1

# Set up an empty array to hole the resampled data:
xmin, ymin, xmax, ymax = 0, 0, 661000, 1241000  # resolution of the existing SHETRAN inputs
new_transform = from_bounds(xmin, ymin, xmax, ymax,
                            width=(xmax - xmin) // resolution_output,
                            height=(ymax - ymin) // resolution_output)
new_shape = ((ymax - ymin) // resolution_output, (xmax - xmin) // resolution_output)
resampled_lake_raster = np.empty(new_shape)

# Resample the data to the desired resolution using the average value in each cell:
reproject(  # You could also do this by applying the row_difference and cell_reduce method from the DEM.
    source=reclassified_lake_data, destination=resampled_lake_raster, src_transform=merged_transform,
    src_crs="EPSG:27700", dst_transform=new_transform, dst_crs="EPSG:27700",
    resampling=Resampling.average  # Average value (0s & 1s)
)

# Convert cells that are over the lake threshold to 1s those below to -9999:
lake_threshold = 0.25  # If the threshold is over 0.5 then you can use Resampling.mode above instead
resampled_lake_raster[resampled_lake_raster>=lake_threshold] = 1
resampled_lake_raster[resampled_lake_raster<lake_threshold] = -9999

# Write as an asc file:
output_path = f'{root}/Processed Data/UK_Lake_Mask_{resolution_output}m'

# Save to file
with rasterio.open(
        output_path+'.asc', "w", driver="AAIGrid", height=resampled_lake_raster.shape[0], width=resampled_lake_raster.shape[1],
        count=1, dtype=resampled_lake_raster.dtype, crs="EPSG:27700", transform=new_transform, nodata=-9999) as dst:
    dst.write(resampled_lake_raster, 1)

## Subsurface Soil and Aquifer Data

The following code snippets will create the soil lookup maps for the UK, these detail the makeup of the subsurface. Our aim is to make a raster map where each cell represents a subsurface column type (e.g. soil type + aquifer type, with depths), along with a csv file that contains the depths and parameters for each column type. This final raster map (.asc) and details table (.csv) will be made up of various spatial layers, typically a soil and a geology, with options for multiple soil layers, different geology types, and superficial deposits.

The data we use is typically split into a file that has the subsurface layer type and another that has the depth at the base of the layer. 

We will need to create some of these layers and do some reformatting before we use them.

The later steps in this process will create a mask over the UK, and then sample the raster data layers above, then convert these lookup values into their actual soil types, parameters and depths, before finally restructuring these into a soil properties map and data table that we use to build the SHETRAN models. You will need to repeat this process to build setups at different resolutions.

Several datasets are used in this process - these are in the form of shapefiles or raster files with lookup tables. You can add more, but the basic ones are processed below. These are:

<br><br>

Dataset Name | Format | Type | Source | Link | Notes
------------ | ------ | ---- |------- | ---- | -----
European Soil Database | Rasters with lookup tables | Soil types and depths | JRC European Soil Data Centre | https://esdac.jrc.ec.europa.eu/content/european-soil-database-v2-raster-library-1kmx1km | Surface and subsurface soil types / depths. Many other datasets available.
digmap625_superficial_arc | Shapefile | Superficial deposit types | BGS| https://www.bgs.ac.uk/datasets/bgs-geology-625k/ | The superficial deposits map has missing data in the west of NI - as such, it is recomended that the layer is edited so that there is no data over NI.
BGS Superficial deposit thickness model 1kmHEX | Shapefile| Superficial deposit depths | BGS | https://www.bgs.ac.uk/datasets/superficial-thickness-model-1-km-hex-grid/ | This is used alongside the superficial deposit maps (digmap625_superficial_arc).<br><br>The thickness map has no data for NI and the Isle of Man.<br><br>Higher resolution thickness data is available through the ASTM: https://www.data.gov.uk/dataset/09c92f49-1cbc-4329-b7c0-6dc3c324ec04/national-superficial-deposits-thickness-model-sdtm.
Hydrogeology | Shapefile | Hydrogeological productivity | BGS | https://www.bgs.ac.uk/datasets/hydrogeology-625k/ | This has previously been combined with teh Aquifer Property Manual for properties.
Geology | Shapefiles | Bedrock geology | BGS | 625k_V5_BEDROCK_Geology_Polygons | https://www.bgs.ac.uk/datasets/bgs-geology-625k/ | -

<br>

**Notes**
- The BGS superficial geology map is incomplete over NI (2025). It is recommended that it is edited prior to use to remove data over NI for clarity. The Thickness data also only covers GB.
- The resampling will take the shape file feature that is at the centroid of each raster cell, rather than the area dominant feature.

In [ ]:
## This code chunk was the first iteration of reformatting the hydrogeolopgy data map - the code in subsequent chunks does a very similar process as part of a loop that can be used alongside other datasets.

# # 1. Load the hydrogeology Data and dissolve to group entries with the same information:
# hydro_shape = gpd.read_file("I:/SHETRAN_GB_2021/07_GIS/hydrogeology625k/HydrogeologyUK_IoM_v5.shp")

# # 1b. Format the Low Productivity Names as there are two that do not match: Check with list(set(hydro_shape['CHARACTER']))
# hydro_shape['CHARACTER'].replace('Low productive aquifer', 'Low productivity aquifer', inplace=True)
# hydro_shape['CHARACTER'].replace('Low productive aquifer', 'Low productivity aquifer', inplace=True)
# hydro_shape['FLOW_MECHA'].fillna('No flow mechanism', inplace=True)

# # 2. Give each rock unit an ID:
# hydro_shape_dissolved = hydro_shape.dissolve(['ROCK_UNIT', 'CLASS', 'CHARACTER', 'FLOW_MECHA', 'SUMMARY'], as_index=False, dropna=False)
# hydro_shape_dissolved['Raster_ID'] = np.arange(1, hydro_shape_dissolved.shape[0]+1)

# # 3. Convert IDs into a raster of desired resolution and correct extents:
# # 3a. Define parameters
# bounds = (0, 0, 661000, 1241000)  # (x_min, y_min, x_max, y_max)
# # resolution = 50  # Resolution in meters
# no_data_value = -9999

# # 3b. Calculate raster dimensions
# width = int((bounds[2] - bounds[0]) / resolution_output)  # Columns
# height = int((bounds[3] - bounds[1]) / resolution_output)  # Rows
# transform = rasterio.transform.from_bounds(*bounds, width, height)

# # 3b. Rasterize the shapefile
# shapes = ((geom, value) for geom, value in zip(hydro_shape_dissolved.geometry, hydro_shape_dissolved['Raster_ID']))
# raster_data = rasterio.features.rasterize(shapes, out_shape=(height, width), transform=transform, fill=no_data_value, dtype="float32",)

# # Step 5: Save the raster to an ASCII file
# with rasterio.open(
#         f'{root}/Processed Data/APM Raster {resolution_output}m.asc', "w", driver="AAIGrid", height=height,
#         width=width, count=1, dtype="float32", crs=hydro_shape_dissolved.crs,  # Use CRS from shapefile
#         transform=transform, nodata=no_data_value) as dst:
#     dst.write(raster_data, 1)

# # 4. Write the raster and the technical data linked to each ID:
# hydro_shape_dissolved[['Raster_ID', 'ROCK_UNIT', 'Flow Mechanism', 'SUMMARY']].to_csv(f'{root}/Processed Data/AMP Raster Data Table.csv', index=False)

### Reformat Input Datasets into rasters and lookup tables for Sampling 

This code will run through layers (shapefiles and then rasters) and create a look-up raster and a csv of soil/subsurface structures.

Once we have made these layers, we can then sample them as we desire to build models with the desired layers.

#### Shapefile Datasets 

These are mostly the BGS geology datasets. We will load these in, disolve them, create a raster of the desired extents (the UK), then sample the shapefiles into the raster.

We will take shapefiles of subsurface data and convert them into raster data. After than, we will build a raster mask, where each cell is a soil solumn. We then go through each cell in the mask and sample the soil raster we created earlier, creating a csv lookup table of soil data for each cell.

In [105]:
# 1. convert the soil shapefiles into raster datasets:
shapefile_datasets = [
    # "I:/SHETRAN_GB_2021/07_GIS/digmap625_bedrock_arc/625k_V5_BEDROCK_Geology_Polygons.shp",
    "I:/SHETRAN_GB_2021/07_GIS/digmap625_superficial_arc - GBonly/UK_625k_SUPERFICIAL_Geology_Polygons - GBonly.shp",
    # "I:/SHETRAN_GB_2021/07_GIS/hydrogeology625k/HydrogeologyUK_IoM_v5.shp",
    # "I:/SHETRAN_GB_2021/07_GIS/BGS Superficial deposit thickness model 1kmHEX/SDTM_HEX/Data/ESRI/BGS_SDTM_1km.shp"
    ]

# Convert each shapefile into a raster dataset:
for shapefile_dataset in shapefile_datasets:
    print(shapefile_dataset)

    soil_shape = gpd.read_file(shapefile_dataset)

    # Dissolve to group entries with the same information:
    dissolve_cols = soil_shape.columns.difference(['geometry']).to_list()
    soil_shape_dissolved = soil_shape.dissolve(dissolve_cols, as_index=False, dropna=False)
    soil_shape_dissolved['Raster_ID'] = np.arange(1, soil_shape_dissolved.shape[0]+1)

    # # Remove any commas from strings to avoid issues with writing to csvs:
    # soil_shape_dissolved.replace({',': ''}, inplace=True)

    # # Remove duplicate rows (excluding geometry) [I don't think that there are duplicates here]
    # dedup_cols = soil_shape_dissolved.columns.difference(['geometry', 'Raster_ID']).to_list()
    # soil_shape_dedup = soil_shape_dissolved.drop_duplicates(subset=dedup_cols, keep='first').reset_index(drop=True)
    # # Remap Raster_IDs to be consecutive
    # soil_shape_dedup['Raster_ID'] = np.arange(1, soil_shape_dedup.shape[0]+1)

    # # Build mapping from old Raster_ID to new Raster_ID
    # # For each unique row, map all duplicates to the first occurrence
    # row_tuples = soil_shape_dissolved[dedup_cols].apply(lambda row: tuple(row), axis=1)
    # dedup_row_tuples = soil_shape_dedup[dedup_cols].apply(lambda row: tuple(row), axis=1)
    # tuple_to_new_id = {tup: rid for tup, rid in zip(dedup_row_tuples, soil_shape_dedup['Raster_ID'])}
    # old_to_new_id = [tuple_to_new_id[tup] for tup in row_tuples]

    # Define parameters and calculate raster dimensions
    bounds = (0, 0, 661000, 1241000)  # (x_min, y_min, x_max, y_max)
    no_data_value = -9999
    width = int((bounds[2] - bounds[0]) / resolution_output)  # Columns
    height = int((bounds[3] - bounds[1]) / resolution_output)  # Rows
    transform = rasterio.transform.from_bounds(*bounds, width, height)

    # Rasterize the shapefile
    shapes = ((geom, value) for geom, value in zip(soil_shape_dissolved.geometry, soil_shape_dissolved['Raster_ID']))
    raster_data = rasterize(shapes, out_shape=(height, width), transform=transform, fill=no_data_value, dtype="float32",)

    # Save the raster to an ASCII file
    output_name = os.path.splitext(os.path.basename(shapefile_dataset))[0]
    with rasterio.open(
            f'{root}/Processed Data/Soil_column_datasets/{output_name} Raster {resolution_output}m.asc', 
            "w", driver="AAIGrid", height=height,
            width=width, count=1, dtype="float32", crs=soil_shape_dissolved.crs,  # Use CRS from shapefile
            transform=transform, nodata=no_data_value
            ) as dst:
          dst.write(raster_data, 1)

    # Write the raster and the technical data linked to each ID:
    soil_shape_dissolved.drop(columns='geometry').to_csv(
         f'{root}/Processed Data/Soil_column_datasets/{output_name} Raster Data Table.csv', index=False)

I:/SHETRAN_GB_2021/07_GIS/digmap625_superficial_arc - GBonly/UK_625k_SUPERFICIAL_Geology_Polygons - GBonly.shp


The **hydrogeology** data has a log of columns, we will therefore edit some of these to simplify the lookuop table into just a flow type / productivity and a summary column that contains the rock type and the summary notes. These values were correct at the time of writing but you may wish to check them.

In [3]:
# Edit the Hydrogeology layer so that the data is more condenced for use later:
hydrogeol = pd.read_csv(f'{root}/Processed Data/Soil_column_datasets/HydrogeologyUK_IoM_v5 Raster Data Table.csv')

# Make a column that contains both the Character and the Flow_mechanism:
flow_mech = {
    '1A': 'Highly productive aquifer (intergranular flow)',
    '1B': 'Moderately productive aquifer (intergranular flow)',
    '1C': 'Low productivity aquifer (intergranular flow)',
    '2A': 'Highly productive aquifer (fracture flow)',
    '2B': 'Moderately productive aquifer (fracture flow)',
    '2C': 'Low productivity aquifer (fracture flow)',
    '3': 'Rocks with essentially no groundwater'
    }
hydrogeol['Flow Mechanism'] = hydrogeol['CLASS'].apply(lambda x: flow_mech.get(str(x), np.nan))

# Combine the rock unit and the summary to make a useful notes column:
hydrogeol['Summary'] = hydrogeol['ROCK_UNIT'] + ': ' + hydrogeol['SUMMARY']

# To make it clear which aquifers the hydrogeology names refer to, give each a number:
units = hydrogeol['Summary'].unique()

coutner = 1
for unit in units:
    # Identify matching indexes:
    matches = hydrogeol['Summary'] == unit

    # Build replacement name with number:
    unit_name = f"HGeo{coutner} {hydrogeol.loc[matches, 'Flow Mechanism'].values[0]}"
    # print(unit_name)

    hydrogeol.loc[matches, 'Flow Mechanism'] = unit_name
    coutner += 1

# Write out the data:
hydrogeol.drop(columns=['CHARACTER', 'CLASS', 'FLOW_MECHA', 'OBJECTID', 'ROCK_UNIT', 'VERSION', 'SUMMARY'], inplace=True)
hydrogeol.to_csv(f'{root}/Processed Data/Soil_column_datasets/HydrogeologyUK_IoM_v5 Raster Data Table.csv', index=False)

In [4]:
# Also remove all duplicated entries and reset the raster IDs:
remove_map_df_duplicates(map_path=f'{root}/Processed Data/Soil_column_datasets/HydrogeologyUK_IoM_v5 Raster 1000m.asc',
                         table_path=f'{root}/Processed Data/Soil_column_datasets/HydrogeologyUK_IoM_v5 Raster Data Table.csv',
                         ID_col='Raster_ID', duplicate_columns=['Flow Mechanism', 'Summary'],
                         output_suffix='', data_format='%1.0f')

C:\Users\nbs65\AppData\Local\Temp\ipykernel_444\1103205447.py:71: DeprecationWarning: loadtxt(): Parsing an integer via a float is deprecated.  To avoid this warning, you can:
    * make sure the original data is stored as integers.
    * use the `converters=` keyword argument.  If you only use
      NumPy 1.23 or later, `converters=float` will normally work.
    * Use `np.loadtxt(...).astype(np.int64)` parsing the file as
      floating point and then convert it.  (On all NumPy versions.)
  (Deprecated NumPy 1.23)
  arr = np.loadtxt(file_path, dtype=data_type, skiprows=6)


The **superficial deposit thickness** dataset has a lot of values  with a small % coverage of deposit. We do not want to use these in the model - we will only include superficial deposits in the model where there is >50% coverage (you may want to edit this fraction). In the data table, we will therefore change all depths where coverage is less than 50% to 0.

In [101]:
# Load the superficial thickness dataset:
super_depth = pd.read_csv(f"{root}/Processed Data/Soil_column_datasets/BGS_SDTM_1km Raster Data Table.csv")

# Change the depths with less than majority cover to 0:
super_depth.loc[super_depth['COVER_PCT'] < 0.5, 'BSTM_MAX'] = 0
super_depth.loc[super_depth['COVER_PCT'] < 0.5, 'BSTM_MEAN'] = 0

# Overwrite the dataset - deposits with 0 depths will not be used in later code:
super_depth.drop(columns=['UID', 'VERSION'], inplace=True)
super_depth.to_csv(f"{root}/Processed Data/Soil_column_datasets/BGS_SDTM_1km Raster Data Table.csv", index=False)

remove_map_df_duplicates(map_path=f'{root}/Processed Data/Soil_column_datasets/BGS_SDTM_1km Raster 1000m.asc',
                         table_path=f'{root}/Processed Data/Soil_column_datasets/BGS_SDTM_1km Raster Data Table.csv',
                         ID_col='Raster_ID', duplicate_columns=['BSTM_MAX', 'BSTM_MEAN', 'COVER_PCT'],
                         output_suffix='', data_format='%1.0f')

KeyError: "['UID', 'VERSION'] not found in axis"

**UK_625k_SUPERFICIAL_Geology**

Change the name "CLAY, SILT AND SAND" to CLAY AND SILT AND SAND so that there is not a comma in the name. Also change out of capitals and add 'Superficials' before the name.

In [107]:
superficials = pd.read_csv(f'{root}/Processed Data/Soil_column_datasets/UK_625k_SUPERFICIAL_Geology_Polygons - GBonly Raster Data Table.csv')

superficials['ROCK_D'] = [f"{r.replace(',', '').replace(' (give log description in Comments field)', '').capitalize()} (superficial)" for r in superficials['ROCK_D']]

superficials.to_csv(f'{root}/Processed Data/Soil_column_datasets/UK_625k_SUPERFICIAL_Geology_Polygons - GBonly Raster Data Table.csv', index=False)

#### Raster Datasets

These are currently just the European Soil datasets. We will reformat the European Soil Database raster into a more useful format.

There are lots of datasets in the download, but we will use the following:

Layer Name | Description | Notes
-----------|-------------| -----
TXSRFDO    | Dominant surface textural class of the STU.| 
TXDEPCHG   | Depth class to a textural change of the dominant and/or secondary surface texture of the STU.| Sometimes these values are missing when the soil types are 0 (No information) and so it is important to check the maps once they are made to ensure that there is not missing data in urban areas.
TXSUBDO    | Dominant sub-surface textural class of the STU.| 
DR         | Depth to rock. | Depths taken to be the deepest of the range (not range is not in order in text description). Sometimes these values are missing when the soil types are 0 (No information) and so it is important to check the maps once they are made to ensure that there is not missing data in urban areas.

This gives us the proportions of clay/sand in a topsoil, subsoil, the depth when topsoil changes to subsoil and the depth of the subsoil.


**Manual Processing**

Each layer comes with a .txt file containing the details for each raster value/group. Copy these into csv files of the same name (layer Raster Data Table.csv) with edits to make the columns seperated by commas:

*class*,*texture*<br>
*0*,*No information (maybe urban)*<br>
*1*,*Coarse (18% < clay and > 65% sand)*<br>
*2*,*Medium (18% < clay < 35% and >= 15% sand or 18% <clay and 15% < sand < 65%)*<br>
*3*,*Medium fine (< 35% clay and < 15% sand)*<br>
*4*,*Fine (35% < clay < 60%)*<br>
*5*,*Very fine (clay > 60 %)*<br>
*9*,*No mineral texture (Peat soils)*<br>

MAKE SURE THAT THERE ARE NO SPACES IN THE CSV COLUMN NAMES AFTER THE COMMAS.

Eg. *class*,*texture* not *class*, *texture*

Additional data on the soils can be found here: https://esdac.jrc.ec.europa.eu/ESDB_Archive/eusoils_docs/other/PTRDBprojRepFinal3.pdf


In [69]:
soil_layer_folder = root + "ESDB-Raster-Library-1k-GeoTIFF-20240507/ESDB-Raster-Library-1k-GeoTIFF-20240507/"
soil_layers = ['TXDEPCHG', 'TXSRFDO', 'TXSUBDO', 'DR']

# For each soil layer tiff file, convert to a raster asc file with the correct extents and resolution:
for soil_layer in soil_layers:
    print(soil_layer)

    # Open the soil layer GeoTIFF file
    with rasterio.open(os.path.join(soil_layer_folder, soil_layer, f'{soil_layer}.tif')) as src:
        # Define target parameters
        dst_crs = "EPSG:27700"
        bounds = (0, 0, 661000, 1241000)  # (x_min, y_min, x_max, y_max)
        width = int((bounds[2] - bounds[0]) / resolution_output)  # Columns
        height = int((bounds[3] - bounds[1]) / resolution_output)  # Rows
        transform = rasterio.transform.from_bounds(*bounds, width, height)
        no_data_value = -9999

        # Update metadata for the output raster
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height,
            'nodata': no_data_value,
            'dtype': 'float32'
        })

        # Create an empty array to hold the resampled data
        resampled_raster = np.empty((height, width), dtype='float32')

        # Reproject and resample the raster data
        reproject(
            source=rasterio.band(src, 1),
            destination=resampled_raster,
            src_transform=src.transform,
            src_crs=src.crs,
            dst_transform=transform,
            dst_crs=dst_crs,
            resampling=Resampling.nearest  # Adjust resampling method if needed
        )

        # Ensure that values of 255 are also no data:
        resampled_raster[resampled_raster == 255] = no_data_value

        # Write the resampled raster to an ASCII file
        output_path = os.path.join(root, 'Processed Data', 'Soil_column_datasets', f'{soil_layer} Raster {resolution_output}m.asc')
        with rasterio.open(
                output_path, "w", driver="AAIGrid", height=resampled_raster.shape[0], width=resampled_raster.shape[1],
                count=1, dtype=resampled_raster.dtype, crs=dst_crs, transform=transform, nodata=no_data_value) as dst:
            dst.write(resampled_raster, 1)

TXDEPCHG
TXSRFDO
TXSUBDO
DR


### Resample Re-formatted Datasets into Soil Map/Table

We will now sample the above layers to generate a soil type lookup map (.asc) and soil details table (csv) using the desired input layers. This will sample the layers spatially. This will process the following steps:
1. Make a mask over the UK, this is what we will use the sample the datasets
2. Create a function for sampling the data.
3. Apply the function to the desired datasets - this will build a table that contains the mask cell IDs and then the dataset keys that we've sampled. This should include datasets of the subsurface type and their depths.
4. We will check that all of the table rows include the necessary subsurfaces. Any that are missing these will be dropped. This is to ensure that we only use mask cells that have a full subsurface column.
5. We then add a few columns that will hold the cell IDs to extract Notes from the data tables - these are often the same Cell IDs as are in the soil types.
6. Map the actual data values to the cell lookup values.
7. Perform some cleaning and edits to ensure that the data is working as we expect.
8. Remove duplicated values from the data table and remap the raster map to match.
9. Reformat the data table to match the SHETRAN soil library files, where each cell has an ID and multiple layers.
10. Merge any adjacent soils of the same type and remove any layers that are shallower than the one above. 
11. Remove any duplicate soil column types and remap the Soil Library IDs and the raster map so that the Cell IDs start from 1 and run consequtively.
12. Save the datasets.

In [124]:
# 1 Create a new mask of the set extents:
bounds = (0, 0, 661000, 1241000)  # (x_min, y_min, x_max, y_max)
no_data_value = -9999
width = int((bounds[2] - bounds[0]) / resolution_output)  # Columns
height = int((bounds[3] - bounds[1]) / resolution_output)  # Rows
transform = rasterio.transform.from_bounds(*bounds, width, height)
mask_data = np.arange(1, (width*height)+1).reshape((height, width))
with rasterio.open(
        f'{root}/Processed Data/Soil_column_datasets/Empty_Soil_Column_Mask_{resolution_output}m.asc', "w", driver="AAIGrid", height=height,
        width=width, count=1, dtype="int32", crs="EPSG:27700",  # Use CRS from shapefile
        transform=transform, nodata=no_data_value) as dst:
    dst.write(mask_data, 1)

In [2]:
# 2. Create a function for sampling the raster data:
def sample_raster_to_mask_df(mask_path, data_layers):
    """
    Sample multiple raster layers using a mask raster and return a DataFrame indexed by mask cell values.
    
    Parameters:
        mask_path (str): Path to the mask raster (ASCII grid).
        data_layers (dict): Dictionary of {column_name: data_raster_path}.
        
    Returns:
        pd.DataFrame: DataFrame indexed by Cell_ID with columns for each sampled data layer.
    """
    # import rasterio
    # import numpy as np
    # import pandas as pd

    # Load mask raster
    mask, _, nrows, xll, yll, cellsize, nodata, _, _ = read_ascii_raster(mask_path, data_type=int, replace_NA=False)
    rows, cols = np.where(mask != nodata)

    # Get the coordinates of the center of each mask cell:
    xs, ys = rasterio.transform.xy(
        rasterio.transform.from_origin(xll, yll + nrows * cellsize, cellsize, cellsize),
        rows, cols
    )
    coords = list(zip(xs, ys))
    cell_ids = mask[rows, cols]

    # Prepare DataFrame
    df = pd.DataFrame({'Cell_ID': cell_ids})
    df.set_index('Cell_ID', inplace=True)

    # Sample each data layer
    for colname, raster_path in data_layers.items():
        print(colname)
        with rasterio.open(raster_path) as data_src:
            sampled_vals = [val[0] for val in data_src.sample(coords)]
        df[colname] = sampled_vals

    return df

In [126]:
# 3. Apply the function, adding the mask IDs to a dataframe, with a column for the value in each of the sampled rasters. Include both the soil type and soil depth datasets. Add and remove these to build the soil column of your dreams.
# This is quite slow, but works nicely.
data_folder = os.path.join(root, 'Processed Data', 'Soil_column_datasets')

data_layers = {
    'TXSRFDO': os.path.join(data_folder, data_folder, 'TXSRFDO Raster 1000m.asc'),
    'TXSUBDO': os.path.join(data_folder, data_folder, 'TXSUBDO Raster 1000m.asc'),
    'superficial': os.path.join(data_folder, data_folder, 'UK_625k_SUPERFICIAL_Geology_Polygons - GBonly Raster 1000m.asc'),
    'APM': os.path.join(data_folder, data_folder, 'HydrogeologyUK_IoM_v5 Raster 1000m.asc'),

    'TXSRFDO_depth': os.path.join(data_folder, 'TXDEPCHG Raster 1000m.asc'),
    'TXSUBDO_depth': os.path.join(data_folder, 'DR Raster 1000m.asc'),
    'superficial_depth': os.path.join(data_folder, 'BGS_SDTM_1km Raster 1000m.asc'),
    # basement (e.g. APM) depth set later.
}

mask_path = os.path.join(data_folder, f'Empty_Soil_Column_Mask_{resolution_output}m.asc')
df = sample_raster_to_mask_df(mask_path, data_layers)

TXSRFDO
TXSUBDO
superficial
APM
TXSRFDO_depth
TXSUBDO_depth
superficial_depth


In [127]:
# 4. Check that each mask cell (row) contains the required layers, remove if not: 

# For each row, check whether the required columns contain 'nan' strings and remove them if they do not:
# This is a good way of reducing the data to just the extent of the area you're interested in (i.e. setting all of the sea around the UK to -9999).
required_columns = ['TXSRFDO', 'APM']

for column in required_columns:

    # Get the rows IDs to be removed:
    remove = df[df[column]== -9999].index

    # Drop these rows from the dataframe:
    df = df[~df.index.isin(remove)]

# Later you will want to crop down the mask to remove these unnecessary IDs (set the to -9999).

In [128]:
# 5. Add some columns with IDs that can be used to extract notes from the layers: 
# Add some columns to hold notes, these use the same lookups keys as the soil details:
df['TXSRFDO_notes'] = df['TXSRFDO']
df['TXSUBDO_notes'] = df['TXSUBDO']
df['superficial_notes'] = df['superficial']
df['APM_notes'] = df['APM']

df.head(5)

,TXSRFDO,TXSUBDO,superficial,APM,TXSRFDO_depth,TXSUBDO_depth,superficial_depth,TXSRFDO_notes,TXSUBDO_notes,superficial_notes,APM_notes
Cell_ID,,,,,,,,,,,
15664,9.0,9.0,9.0,42.0,5.0,3.0,80.0,9.0,9.0,9.0,42.0
15667,9.0,9.0,9.0,35.0,5.0,3.0,85.0,9.0,9.0,9.0,35.0
16325,9.0,9.0,9.0,42.0,5.0,3.0,80.0,9.0,9.0,9.0,42.0
16327,9.0,9.0,9.0,35.0,5.0,3.0,39.0,9.0,9.0,9.0,35.0
16328,9.0,9.0,9.0,35.0,5.0,3.0,85.0,9.0,9.0,9.0,35.0


In [129]:
# 6. Fill the data table with actual values. Supply the column name, data table path and Cell ID and cell details columns. Do this for the subsurface type, depth and notes.

# Example: columns in df are ['TXSRFDO', 'TXSUBDO', 'APM']
# Each column contains integer codes that need to be mapped to descriptions from a lookup CSV

# Define the mapping: column name -> (lookup csv path, code column, value column)
lookup_info = {
    'TXSRFDO': (os.path.join(data_folder, 'TXSRFDO Raster Data Table.csv'), 'Class', 'Texture'),
    'TXSUBDO': (os.path.join(data_folder, 'TXSUBDO Raster Data Table.csv'), 'Class', 'Texture'),
    'superficial': (os.path.join(data_folder, 'UK_625k_SUPERFICIAL_Geology_Polygons - GBonly Raster Data Table.csv'), 'Raster_ID', 'ROCK_D'),
    'APM': (os.path.join(data_folder, 'HydrogeologyUK_IoM_v5 Raster Data Table.csv'), 'Raster_ID', 'Flow Mechanism'),

    'TXSRFDO_depth': (os.path.join(data_folder, 'TXDEPCHG Raster Data Table.csv'), 'Class', 'Depth to Change'),
    'TXSUBDO_depth': (os.path.join(data_folder, 'DR Raster Data Table.csv'), 'DR', 'Depth to rock'),
    'superficial_depth': (os.path.join(data_folder, 'BGS_SDTM_1km Raster Data Table.csv'), 'Raster_ID', 'BSTM_MEAN'),
    
    'TXSRFDO_notes': (os.path.join(data_folder, 'TXSRFDO Raster Data Table.csv'), 'Class', 'Notes'),
    'TXSUBDO_notes': (os.path.join(data_folder, 'TXSUBDO Raster Data Table.csv'), 'Class', 'Notes'),
    'superficial_notes': (os.path.join(data_folder, 'UK_625k_SUPERFICIAL_Geology_Polygons - GBonly Raster Data Table.csv'), 'Raster_ID', 'LEX_D'),
    'APM_notes': (os.path.join(data_folder, 'HydrogeologyUK_IoM_v5 Raster Data Table.csv'), 'Raster_ID', 'Summary'),
    # Add more as needed
}

for col, (csv_path, code_col, value_col) in lookup_info.items():
    
    # Load lookup table:
    lut = pd.read_csv(csv_path)
    lut.columns = [c.strip() for c in lut.columns]  # Remove leading/trailing white space.
    
    # Build mapping dictionary:
    code_to_val = pd.Series(lut[value_col].values, index=lut[code_col]).to_dict()
    
    # Map the df column (skip -9999 and nan)
    df[col] = df[col].map(lambda x: code_to_val.get(x, np.nan) if pd.notnull(x) and x != -9999 else np.nan)

# Now df columns contain the mapped values (descriptions) instead of codes.
df.head(5)

,TXSRFDO,TXSUBDO,superficial,APM,TXSRFDO_depth,TXSUBDO_depth,superficial_depth,TXSRFDO_notes,TXSUBDO_notes,superficial_notes,APM_notes
Cell_ID,,,,,,,,,,,
15664,Peat topsoil,Peat subsoil,Peat (superficial),HGeo42 Low productivity aquifer (fracture flow),1.2,0.4,1.0,No mineral texture,No mineral texture,PEAT,APPIN GROUP: Small amounts of groundwater in n...
15667,Peat topsoil,Peat subsoil,Peat (superficial),HGeo35 Low productivity aquifer (fracture flow),1.2,0.4,1.0,No mineral texture,No mineral texture,PEAT,SOUTHERN HIGHLAND GROUP: Small amounts of grou...
16325,Peat topsoil,Peat subsoil,Peat (superficial),HGeo42 Low productivity aquifer (fracture flow),1.2,0.4,1.0,No mineral texture,No mineral texture,PEAT,APPIN GROUP: Small amounts of groundwater in n...
16327,Peat topsoil,Peat subsoil,Peat (superficial),HGeo35 Low productivity aquifer (fracture flow),1.2,0.4,0.0,No mineral texture,No mineral texture,PEAT,SOUTHERN HIGHLAND GROUP: Small amounts of grou...
16328,Peat topsoil,Peat subsoil,Peat (superficial),HGeo35 Low productivity aquifer (fracture flow),1.2,0.4,1.0,No mineral texture,No mineral texture,PEAT,SOUTHERN HIGHLAND GROUP: Small amounts of grou...


In [130]:
# 7. Do some edits to fix some data issues.
# Toggle these on and off as needed:

# --- Base Layer / APM ---
# We do not have base depths for the bottom layer (APM), so lets assign these either as a constant, or as a function of superficial depth as this is often very deep:
df['APM_depth'] = [max(25, depth*2) for depth in df['superficial_depth']]

# --- TXSRFDO / TXDEPCHG ---
# TXSRFDO with 'No Information' do not have depths. Give these depths of 1m:
# Set TXSRFDO_depth to 1 where TXSRFDO is 'No information' and TXSRFDO_depth is NaN
# Set the the other soil if there is an NaN:
df.loc[df['TXSRFDO_depth'].isna(), 'TXSRFDO_depth'] = df.loc[df['TXSRFDO_depth'].isna(), 'TXSUBDO_depth']
df.loc[df['TXSRFDO']=='No information', 'TXSRFDO'] = df.loc[df['TXSRFDO']=='No information', 'TXSUBDO']

df.loc[df['TXSUBDO_depth'].isna(), 'TXSUBDO_depth'] = df.loc[df['TXSUBDO_depth'].isna(), 'TXSRFDO_depth']
df.loc[df['TXSUBDO']=='No information', 'TXSUBDO'] = df.loc[df['TXSUBDO']=='No information', 'TXSRFDO']

# If that doesn't work (i.e. both soils are NaN, then set depths to 1) 
df.loc[df['TXSRFDO_depth'].isna(), 'TXSRFDO_depth'] = 1
df.loc[df['TXSUBDO_depth'].isna(), 'TXSUBDO_depth'] = 1

# --- Superficial / BGS_SDTM_1km ---
# Superficials are not always present - when this is the case, set their depths to 0 so that they are removed.
df.loc[df['superficial'].isna(), 'superficial_depth'] = 0
# df.loc[df['superficial'].isna(), 'superficial'] = 'No superficial deposit'  # not needed

# # Remove commas from all string entries to avoid issues with writing to CSV:
# df.replace({',': ''}, inplace=True)

In [131]:
# 8. Find duplicates and get mapping from duplicate index to original index - this might be able to be done using remove_map_df_duplicates.
dupes = df.duplicated(keep='first')
df_reset = df.reset_index()

# Find the first occurrence for each duplicate row
# first_occurrence = df_reset[dupes.index[dupes]].drop_duplicates()
# Map each row to its first occurrence using a hashable tuple
row_tuples = df.apply(lambda row: tuple(row), axis=1)
first_idx_map = {}
seen = {}
for idx, tup in zip(df.index, row_tuples):
    if tup not in seen:
        seen[tup] = idx
    first_idx_map[idx] = seen[tup]

# Remap mask IDs

# Load the empty mask (Empty_Soil_Column_Mask) created above:
mask, _, _, _, _, _, _, _, _ = read_ascii_raster(mask_path, data_type=int, replace_NA=False)
mask_flat = mask.flatten()
mask_flat = np.array([first_idx_map.get(val, val) for val in mask_flat])

# 3. Remove duplicate rows
df_unique = df[~dupes]

# 4. Reshape mask
mask_unique = mask_flat.reshape(mask.shape)
df_unique.head()

,TXSRFDO,TXSUBDO,superficial,APM,TXSRFDO_depth,TXSUBDO_depth,superficial_depth,TXSRFDO_notes,TXSUBDO_notes,superficial_notes,APM_notes,APM_depth
Cell_ID,,,,,,,,,,,,
15664,Peat topsoil,Peat subsoil,Peat (superficial),HGeo42 Low productivity aquifer (fracture flow),1.2,0.4,1.0,No mineral texture,No mineral texture,PEAT,APPIN GROUP: Small amounts of groundwater in n...,25.0
15667,Peat topsoil,Peat subsoil,Peat (superficial),HGeo35 Low productivity aquifer (fracture flow),1.2,0.4,1.0,No mineral texture,No mineral texture,PEAT,SOUTHERN HIGHLAND GROUP: Small amounts of grou...,25.0
16327,Peat topsoil,Peat subsoil,Peat (superficial),HGeo35 Low productivity aquifer (fracture flow),1.2,0.4,0.0,No mineral texture,No mineral texture,PEAT,SOUTHERN HIGHLAND GROUP: Small amounts of grou...,25.0
16330,Peat topsoil,Peat subsoil,Peat (superficial),HGeo31 Low productivity aquifer (fracture flow),1.2,0.4,1.0,No mineral texture,No mineral texture,PEAT,"UNNAMED IGNEOUS INTRUSION, LATE SILURIAN TO EA...",25.0
16985,Peat topsoil,Peat subsoil,Peat (superficial),HGeo42 Low productivity aquifer (fracture flow),1.2,0.4,0.0,No mineral texture,No mineral texture,PEAT,APPIN GROUP: Small amounts of groundwater in n...,25.0


Now create the soil library file.

This has the following format:

Soil Category| Soil Layer | Soil Type | Depth at base of layer (m) | Saturated Water Content | Residual Water Content | Saturated Conductivity (m/day) | vanGenuchten- alpha (cm-1) | vanGenuchten-n
--- | --- | --- | --- | --- | --- | --- | --- | ---
0|1|DEFAULT_SOIL_CHECK_LOCALLY|1|0.403|0.025|50|0.0383|1.3774
0|2|DEFAULT_LOW_PRODUCTIVITY_GEOLOGY_CHECK_LOCALLY|21|0.3|0.2|0.001|0.1|5
1|1|Medium(18%:clay:35%And:15%sandOr18%:clayAnd15%:sand:65%)|1|0.439|0.01|12.061|0.0314|1.1804
1|2|APM5&6_Low_productivity_aquifer_through_pores_or_cracks|21|0.3|0.2|0.001|0.01|5
2|1|Medium(18%:clay:35%And:15%sandOr18%:clayAnd15%:sand:65%)|1|0.439|0.01|12.061|0.0314|1.1804
2|2|MediumFine(:35%clayand:15%sand)|1.2|0.412|0.01,4|0.0082|1.2179
2|3|APM5&6_Low_productivity_aquifer_through_pores_or_cracks|21.2|0.3|0.2|0.001|0.01|5


Rerun this code using the layers that you're interested in, chaning the output name later on.

In [132]:
# 9. Build the soil library as a list of lists for speed.
# Each inner list represents a row in the final DataFrame.

layer_dict = {1: 'TXSRFDO', 2: 'TXSUBDO', 3: 'superficial', 4: 'APM'}  # Add other layers as needed

soil_library_data = []

for row in range(len(df_unique)):
    cell_id = df_unique.index[row]

    # Loop through each soil layer for the current cell
    for layer_num, layer_name in layer_dict.items():

        # Get soil type, depth, and notes for this cell/layer
        soil_type = df_unique.iloc[row][layer_name]
        soil_depth = df_unique.iloc[row][f'{layer_name}_depth']
        
        # soil_depth = df_unique[0] if len(soil_depth) > 0 else np.nan
        soil_note = df_unique.iloc[row][f'{layer_name}_notes']

        # Append all values as a list (much faster than using dicts or concat)
        soil_library_data.append([
            cell_id,
            int(layer_num),
            soil_type,
            soil_depth,
            np.nan,  # Saturated Water Content
            np.nan,  # Residual Water Content
            np.nan,  # Saturated Conductivity (m/day)
            np.nan,  # vanGenuchten- alpha (cm-1)
            np.nan,  # vanGenuchten-n
            soil_note
        ])

# Convert the list of lists to a DataFrame in one go (very fast)
soil_library = pd.DataFrame(
    soil_library_data,
    columns=[
        'Soil Category',
        'Soil Layer',
        'Soil Type',
        'Depth at base of layer (m)',
        'Saturated Water Content',
        'Residual Water Content',
        'Saturated Conductivity (m/day)',
        'vanGenuchten- alpha (cm-1)',
        'vanGenuchten-n',
        'Notes'
    ]
)

soil_library.head(25)
# Now the soil_library is ready for further processing and saving.

,Soil Category,Soil Layer,Soil Type,Depth at base of layer (m),Saturated Water Content,Residual Water Content,Saturated Conductivity (m/day),vanGenuchten- alpha (cm-1),vanGenuchten-n,Notes
0,15664,1,Peat topsoil,1.2,NaN,NaN,NaN,NaN,NaN,No mineral texture
1,15664,2,Peat subsoil,0.4,NaN,NaN,NaN,NaN,NaN,No mineral texture
2,15664,3,Peat (superficial),1.0,NaN,NaN,NaN,NaN,NaN,PEAT
3,15664,4,HGeo42 Low productivity aquifer (fracture flow),25.0,NaN,NaN,NaN,NaN,NaN,APPIN GROUP: Small amounts of groundwater in n...
4,15667,1,Peat topsoil,1.2,NaN,NaN,NaN,NaN,NaN,No mineral texture
5,15667,2,Peat subsoil,0.4,NaN,NaN,NaN,NaN,NaN,No mineral texture
6,15667,3,Peat (superficial),1.0,NaN,NaN,NaN,NaN,NaN,PEAT
7,15667,4,HGeo35 Low productivity aquifer (fracture flow),25.0,NaN,NaN,NaN,NaN,NaN,SOUTHERN HIGHLAND GROUP: Small amounts of grou...
8,16327,1,Peat topsoil,1.2,NaN,NaN,NaN,NaN,NaN,No mineral texture
9,16327,2,Peat subsoil,0.4,NaN,NaN,NaN,NaN,NaN,No mineral texture


In [134]:
# 10. Clean the dataset by merging adjacent soils of the same type and remove any layers that are shallower than the one above.
def clean_soil_library(soil_library_df, SHETRAN_Max_Depth=100):
    """
    Clean the soil library DataFrame by merging adjacent layers of the same soil type
    and removing layers that are shallower than the one above.
    If the model goes deeper than 100m then the layer that crosses this depth threshold will be set to 99m deep and deeper layers removed. This is because many SHETRAN versions have a maximum depth of 100m.
    """
    cleaned_rows = []
    for cat, group in soil_library_df.groupby('Soil Category'):
        group = group.sort_values('Soil Layer')
        rows = group.to_dict('records')
        i = 0

        # Check through the rows:
        while i < len(rows) - 1:
            layer = rows[i]
            layer_below = rows[i + 1]

            # If the current layer is > 100m deep...
            if float(rows[i]['Depth at base of layer (m)']) > SHETRAN_Max_Depth:
                # ...then set this to 100m:
                rows[i]['Depth at base of layer (m)'] = SHETRAN_Max_Depth
                # Keep only up to the current layer:
                rows = rows[:i + 1]
                # Finish the loop:
                break

            # If the current layer (soil type and notes) is duplicated with the one below...
            if (layer['Soil Type'] + layer['Notes'] == layer_below['Soil Type'] + layer_below['Notes']) or (  
                # ... or the depth does not increase with layer number...
                float(layer['Depth at base of layer (m)']) >= float(layer_below['Depth at base of layer (m)'])):
                # Remove lower layer (layer_below), keeping the maximum of the two depths:
                rows[i]['Depth at base of layer (m)'] = max(float(rows[i]['Depth at base of layer (m)']), float(rows[i+1]['Depth at base of layer (m)']) )
                # Remove the layer below from the list of rows:
                rows.pop(i + 1)

            # Finally, if the layer below is above the maximum depth...
            elif float(layer_below['Depth at base of layer (m)']) > SHETRAN_Max_Depth:
                # ...set its depth to the max specified depth:
                rows[i+1]['Depth at base of layer (m)'] = SHETRAN_Max_Depth
                # Drop any layers below this one:
                rows = rows[:i + 2]  
                # Stop processing further layers
                break

            # If there weren't any issues, carry on to the next layer:
            else:
                i += 1
                
        # Reassign Soil Layer numbers
        for idx, row in enumerate(rows, 1):
            row['Soil Layer'] = idx
            cleaned_rows.append(row)
    return pd.DataFrame(cleaned_rows, columns=soil_library_df.columns)

# Clean the library file of :
soil_library_cleaned = clean_soil_library(soil_library)

In [135]:
# soil_library.head(30)
soil_library_cleaned.head(5)

,Soil Category,Soil Layer,Soil Type,Depth at base of layer (m),Saturated Water Content,Residual Water Content,Saturated Conductivity (m/day),vanGenuchten- alpha (cm-1),vanGenuchten-n,Notes
0,15664,1,Peat topsoil,1.2,NaN,NaN,NaN,NaN,NaN,No mineral texture
1,15664,2,HGeo42 Low productivity aquifer (fracture flow),25.0,NaN,NaN,NaN,NaN,NaN,APPIN GROUP: Small amounts of groundwater in n...
2,15667,1,Peat topsoil,1.2,NaN,NaN,NaN,NaN,NaN,No mineral texture
3,15667,2,HGeo35 Low productivity aquifer (fracture flow),25.0,NaN,NaN,NaN,NaN,NaN,SOUTHERN HIGHLAND GROUP: Small amounts of grou...
4,16327,1,Peat topsoil,1.2,NaN,NaN,NaN,NaN,NaN,No mineral texture


In [136]:
#  11. Remove any duplicated soil column types and remap so that the map indexes start at 1.

# 1. Get the unique valid soil categories from soil_library_cleaned (excluding -9999)
unique_ids = np.unique(soil_library_cleaned['Soil Category'])
# unique_ids = unique_ids[unique_ids != -9999]

# 2. Create a mapping from old ID to new consecutive ID starting from 1
id_map = {old_id: new_id for new_id, old_id in enumerate(unique_ids, start=1)}

# 3. Remap the Soil Category in soil_library_cleaned
soil_library_cleaned['Soil Category'] = soil_library_cleaned['Soil Category'].map(id_map)

# 4. Remap the mask using the same mapping
mask_reset = np.where(
    np.isin(mask_unique, list(id_map.keys())),
    np.vectorize(id_map.get)(mask_unique),
    -9999
)

# soil_library_cleaned
# Now mask_reset and soil_library_cleaned['Soil Category'] are consecutive and matching, starting from 1

### Add Parameters to the Soil Table

Assign soil properties from a csv table. It is simplest to have one of these that is appropriate for all of the layers you use, with simple, identifiable names matching accross the layers. For example:

There are lots of parameters online e.g. *Gupta et al. (2020) SoilKsatDB: global compilation of soil saturated hydraulic conductivity measurements for geoscience applications*.

Dictionary of Subsurface Parameters csv is availble on the GitHub.

In [137]:
# Load in the subsurface parameters to fill in the gaps in the soil_library_cleaned:
parameters = pd.read_csv("I:/SHETRAN_GB_2021/01_Scripts/Other/OFFLINE Generic Catchment Setup Script/Dictionary of Subsurface Parameters.csv")

# Fill the missing values by matching the Soil Types in the two tables:
for param in [
    'Saturated Water Content',
    'Residual Water Content',
    'Saturated Conductivity (m/day)',
    'vanGenuchten- alpha (cm-1)',
    'vanGenuchten-n'
]:
    param_map = pd.Series(parameters[param].values, index=parameters['Soil Type']).to_dict()
    soil_library_cleaned[param] = soil_library_cleaned.apply(
        lambda row: param_map.get(row['Soil Type'], row[param]) if pd.isnull(row[param]) else row[param],
        axis=1
    )

In [138]:
# Check for any remaining NaNs in the parameter columns:
soil_library_cleaned[soil_library_cleaned['Saturated Water Content'].isna()]

,Soil Category,Soil Layer,Soil Type,Depth at base of layer (m),Saturated Water Content,Residual Water Content,Saturated Conductivity (m/day),vanGenuchten- alpha (cm-1),vanGenuchten-n,Notes


In [ ]:
# 12. Save mask_reset as ASCII grid
output_path = 'I:/SHETRAN_GB_2021/02_Input_Data/01 - National Data Inputs for SHETRAN UK/Processed Data/'
file_identifier = 'ESD_BGSsuper_BGShydrogeol'

write_ascii(
    array=mask_reset,
    ascii_ouput_path=os.path.join(output_path, f'UK_Subsurface_{file_identifier}_{resolution_output}m.asc'),
    xllcorner=transform[2],
    yllcorner=transform[5] - (mask_reset.shape[0] * transform[0]),
    cellsize=transform[0],
    NODATA_value=-9999,
    data_format='%d'  # , data_format='%1.0f'
)

# Save soil_library_cleaned as CSV
soil_library_cleaned.to_csv(os.path.join(output_path, f'UK_Subsurface_{file_identifier}_{resolution_output}m.csv'), index=False)

It is recomended that when you build these datasets you note down in a readme what the input data is.

### BGS 3D Geology
This data is more complex and contains depths as well as rock types.
TODO: Request this data when you find the information leaflet.

## Parameter CSVs

### Soil Details
These are the default parameters given to soils and rocks. The datasets is large and so is copied from an existing csv.

### Vegetation Details

The vegetation details csv holds the default parameters for the different land use classes. These are displayed below:

| Veg Type # | Vegetation Type | Canopy storage capacity (mm) | Leaf area index | Maximum rooting depth (m) | AE/PE at field capacity | Strickler overland flow coefficient |
|------------|-----------------|-------------------------------|-----------------|----------------------------|-------------------------|-------------------------------------|
| 1          | Arable          | 1                             | 0.8             | 0.8                        | 0.6                     | 0.6                               |
| 2          | BareGround      | 0                             | 0               | 0.1                        | 0.4                     | 3                                 |
| 3          | Grass           | 1.5                           | 1               | 1                          | 0.6                     | 0.5                               |
| 4          | DeciduousForest | 5                             | 1               | 1.6                        | 1                       | 1                                 |
| 5          | EvergreenForest | 5                             | 1               | 2                          | 1                       | 0.25                              |
| 6          | Shrub           | 1.5                           | 1               | 1                          | 0.4                     | 2                                 |
| 7          | Urban           | 0.3                           | 0.3             | 0.5                        | 0.4                     | 5                                 |
| 8          | Water           | 0                             | 0               | 0.1                        | 0.4                     | 3                                 |
| 11         | UDM_MM2017_Rural| 3                             | 1               | 1                          | 0.63                    | 2                                 |
| 12         | UDM_MM2017_Urban_CombinedSewers| 0.3           | 0.3             | 0.5                        | 1                       | 12                                |
| 13         | UDM_Future_Urban_SeperatedSewers| 0.3          | 0.3             | 0.5                        | 1                       | 12                                |



In [4]:
# Create a dataframe of the above data:
df = pd.DataFrame([
    ['Veg Type #', 'Vegetation Type', 'Canopy storage capacity (mm)', 'Leaf area index',
     'Maximum rooting depth (m)', 'AE/PE at field capacity', 'Strickler overland flow coefficient'],
    [1, 'Arable', 1, 0.8, 0.8, 0.6, 0.6],
    [2, 'BareGround', 0, 0, 0.1, 0.4, 3],
    [3, 'Grass', 1.5, 1, 1, 0.6, 0.5],
    [4, 'DeciduousForest', 5, 1, 1.6, 1, 1],
    [5, 'EvergreenForest', 5, 1, 2, 1, 0.25],
    [6, 'Shrub', 1.5, 1, 1, 0.4, 2],
    [7, 'Urban', 0.3, 0.3, 0.5, 0.4, 5],
    [8, 'Water', 0, 0, 0.1, 0.4, 3],
    [11, 'UDM_MM2017_Rural', 3, 1, 1, 0.63, 2],
    [12, 'UDM_MM2017_Urban_CombinedSewers', 0.3, 0.3, 0.5, 1, 12],
    [13, 'UDM_Future_Urban_SeperatedSewers', 0.3, 0.3, 0.5, 1, 12],
])

# Write the DataFrame to a CSV file
csv_file = f'{root}/Processed Data/Vegetation_Details.csv'
df.to_csv(csv_file, index=False, header=False)

Data has been written to I:/SHETRAN_GB_2021/02_Input_Data/National Data Inputs for SHETRAN UK//Processed Data/vegetation_data.csv


## Move Data into Input Folder

The data has so far been written to a Processed Data folder. This code chunk will move it from there into a new folder that is used to create catchment files.

In [38]:
import shutil

folder_root = 'I:/SHETRAN_GB_2021/02_Input_Data'
# resolutions = ['100m', '200m', '500m', '1000m']
resolutions = ['1000m']

for resolution in resolutions:
    print(resolution)
    source_folder = f'{root}/Processed Data/'
    destination_folder = f'{folder_root}/00 - Raw ASCII inputs for SHETRAN UK/{resolution}_v2'

    filenames = {
        # 'Land_Use': [f'UK Land Use {resolution}.asc', 'SHETRAN_UK_LandCover.asc'],
        # 'DEM': [f'National_OS50_DEM_{resolution}.asc', 'SHETRAN_UK_DEM.asc'],
        # 'Min_DEM': [f'National_OS50_minDEM_{resolution}.asc' , 'SHETRAN_UK_minDEM.asc'],
        # 'Lakes': [f'UK_Lake_Mask_{resolution}.asc', 'SHETRAN_UK_lake_presence.asc'],
        'Soil table': ['UK_Subsurface_ESD_BGSsuper_APM_1000m.csv', 'SHETRAN_UK_Subsurface_ESD_BGSsuper_HydroGeo.csv'],
        'Soil map': ['UK_Subsurface_ESD_BGSsuper_APM_1000m.asc', 'SHETRAN_UK_Subsurface_ESD_BGSsuper_HydroGeo.asc'],
    }

    for file in filenames.keys():
        print('--> ', file)
        shutil.copy2(os.path.join(source_folder, filenames[file][0]),
                     os.path.join(destination_folder, filenames[file][1]))

1000m
-->  Soil table
-->  Soil map


## Superceeded LCM 2007 Code

The following code was used in testing, but not used in the final processing. Potentially due to running too slowly, such as with the vector LCM processing.

Processing steps:
1. Download data (manually, unzip if necessary).
2. Merge data classes as per the table below and save the updated shapefiles.
a. Load each regional shapefile in turn.
b. Remove the unnecessary data.
c. Dissolve the polygons to reduce the files size.
d. Write the shapefiles (these can be removed once the rest of these steps are completed).
3. Merge the shapefiles into a single UK wide vector dataset.
4. Read the UK dataset and resample into the desired resolution and write as asc files. This has the following steps:
a. Create a vector grid of the desired resolution covering the standard SHETRAN UK domain and give each cell an ID.
b. Intersect this grid with the UK land cover data so that each polygon is within a single cell boundary and has the grid ID that it is within.
c. Calculate the area of each intersected polygon, filter using the area and grid cell ID, and remove duplicates, leaving only a single polygon per grid cell (the one with the largest area).
d. Join these polygons back to the original grid (so that the data can be displayed as a regular grid, rather than 1 irregular polygon per grid cell).
e. Rasterise and save the data

In [29]:
# # >>> STEP 2 <<<
# # Define the reclassification dictionary
# reclass_dict = {  # (CEH LCM to SHETRAN Classes)
#     1: 4, 2: 5, 3: 1,
#     4: 3, 5: 3, 6: 3, 7: 3,8: 3,
#     9: 6, 10: 6, 11: 6, 12: 6, 13: 6,
#     14: 2, 15: 2, 16: 2,  17: 2,  18: 2,  19: 2, 20: 2, 21: 2,
#     22: 7, 23: 7
# }
#
# # List the shapefiles in GB:
# GB_LCM = os.path.join(root, 'Land Use Inputs/LCM_2007_vector_GB_Digimap/lcm-2007-vec_5779248')
# GB_LCM_files = os.listdir(GB_LCM)
# shapefiles = [os.path.join(GB_LCM, sf) for sf in GB_LCM_files if sf.endswith('.shp')]
#
# NI_LCM = os.path.join(root, 'Land Use Inputs/LCM_2007_vector_NI_Digimap/lcm-2007-vec-ni_4578539')
# NI_LCM_files = os.listdir(NI_LCM)
# shapefiles.append([os.path.join(NI_LCM, sf) for sf in NI_LCM_files if sf.endswith('.shp')])
#
# # Run through the files (including NI):
# counter = 1
# for shapefile in shapefiles:
#     print(counter, '/', len(shapefiles))
#
#     # Read in the data:
#     sf = gpd.read_file(shapefile)
#
#     # Reclassify from LCM to SHETRAN classes'
#     sf['SHETRAN'] = sf['INTCODE'].map(reclass_dict)
#
#     # Reproject the Northern Ireland file into BNG (from ING):
#     if 'LCM_2007_vector_NI_Digimap' in shapefile:
#         sf = sf.to_crs(epsg=27700)
#
#     # Cull the columns you don't need:
#     columns = sf.columns
#     columns = [column for column in columns if column not in ['SHETRAN', 'geometry']]
#     sf.drop(columns, inplace=True, axis=1)
#
#     # Dissolve the polygons to reduce file size:
#     sf_dissolved = sf.dissolve('SHETRAN')
#
#     # Save the updated shapefile:
#     sf_dissolved.to_file(
#         os.path.join(root, "Land Use Inputs/Reclassified shapefiles", os.path.basename(shapefile))
#     )
#
#     counter += 1

In [ ]:
# The projection files again don't quite match so I have manually copied the projection from the GB files to the NI file... This is a very small difference and does not seem to make any difference to the polygon locations.
#
# Original: PARAMETER["Scale_Factor",0.9996012717]
#
# Updated: PARAMETER["Scale_Factor",0.999601272]

# # >>> Step 3 <<<
# # List the shapefiles in GB:
# shapefile_path = os.path.join(root, 'Land Use Inputs/Reclassified shapefiles')
# shapefiles = os.listdir(shapefile_path)
# shapefiles = [os.path.join(shapefile_path, sf) for sf in shapefiles if sf.endswith('.shp')]
#
# # Merge into a single file:
# gdfs = []
# for shapefile in shapefiles:
#     gdfs.append(gpd.read_file(shapefile))
#
# # Merge all GeoDataFrames into one
# merged_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))
#
# # Save the merged GeoDataFrame to a new shapefile
# merged_gdf.to_file(shapefile_path + '/LCM_2007_vector_UK_BNG.shp')

In [ ]:
# >>> Step 4 <<<
# The following code was used to process the vector LCM straight into raster form. I believe that it works, but is extremely slow (and did not finish when applied nationally."""

# from rasterio.merge import merge
# from rasterio.transform import from_bounds
#
# # Paths for your rasters
# raster_GB_LCM = root + "/Land Use Inputs/LCM 2007 25m Raster/data/lcm2007gb25m.tif"
# raster_NI_LCM = NI_LCM_BNG
#
# # Open LCM GB
# with rasterio.open(raster_GB_LCM) as raster:
#     raster_GB_array = raster.read(1)
#     transform_GB = raster.transform
#     meta_GB = raster.meta.copy()
#
# # Open LCM NI (in BNG)
# with rasterio.open(raster_GB_LCM) as raster:
#     raster_NI_array = raster.read(1)
#     transform_NI = raster.transform
#     meta_NI = raster.meta.copy()
#
# # Merge the two rasters
# merged_raster, merged_transform = merge([(raster_NI_array, transform_NI), (raster_GB_array, transform_GB)])
#
# # Regrid to 50m resolution
# xmin, ymin, xmax, ymax = 0, 0, 661000, 1241000  # Specified extent
# new_transform = from_bounds(xmin, ymin, xmax, ymax, width=(xmax - xmin) // 50, height=(ymax - ymin) // 50)
#
# new_shape = ((ymax - ymin) // 50, (xmax - xmin) // 50)
# resampled_raster = np.empty(new_shape, dtype=merged_raster.dtype)
#
# reproject(
#     source=merged_raster,
#     destination=resampled_raster,
#     src_transform=merged_transform,
#     src_crs="EPSG:27700",
#     dst_transform=new_transform,
#     dst_crs="EPSG:27700",
#     resampling=Resampling.mode  # Or Resampling.average for continuous data
# )
#
# # Save to file
# output_path = "merged_resampled_50m.tif"
# with rasterio.open(
#     output_path,
#     "w",
#     driver="GTiff",
#     height=resampled_raster.shape[0],
#     width=resampled_raster.shape[1],
#     count=1,
#     dtype=resampled_raster.dtype,
#     crs="EPSG:27700",
#     transform=new_transform
# ) as dst:
#     dst.write(resampled_raster, 1)
#
# print(f"Resampled raster saved to {output_path}")


In [ ]:
# # >>> Step 4 <<<
# # Load the vector data (merged shapefile)
# gdf = gpd.read_file(shapefile_path + '/LCM_2007_vector_UK_BNG.shp')
# xmin, ymin, xmax, ymax = 0, 0, 661000, 1241000  # British National Grid boundaries
#
# # TEST DATASET
# # gdf = gpd.read_file(shapefile_path + '/nk_land_parcel.shp')
# # xmin, ymin, xmax, ymax = 399500, 822000, 414500, 868000
#
# # Step 2: Create a vector grid
# cell_size = resolution_output  # 100m resolution
# cols = np.arange(xmin, xmax, cell_size)
# rows = np.arange(ymin, ymax, cell_size)
#
# grid_cells = []
# for x in cols:
#     for y in rows:
#         grid_cells.append(box(x, y, x + cell_size, y + cell_size))
#
# # Turn this into a geodataframe and give it an ID
# grid = gpd.GeoDataFrame({"geometry": grid_cells}, crs=gdf.crs)
# grid['ID'] = np.arange(0, grid.shape[0])
#
# # Step 1: Intersect the grid and the shapefile
# intersected = gpd.overlay(grid, gdf, how='intersection', keep_geom_type=False)
#
# # Step 2: Calculate the area of each intersected polygon
# intersected["area"] = intersected.area
#
# # Step 3: Sort the intersected DataFrame by 'ID' and 'area' and crop to only the largest land type per cell:
# intersected_sorted = intersected.sort_values(by=["ID", "area"], ascending=[True, False])
#
# # Step 4: Drop duplicates based on 'ID', keeping only the largest area
# filtered_intersected = intersected_sorted.drop_duplicates(subset="ID")
# # filtered_intersected.to_file(shapefile_path + '/filtered_intersected.shp')
#
# # 5. Converting filtered_intersected straight to raster misses cells, instead join the LC classes back to the grid:
# # Perform the left join on the 'ID' column
# grid_with_intersected = grid.merge(filtered_intersected[['SHETRAN', 'ID']], on="ID", how="left", suffixes=('_grid', '_intersected'))
# # grid_with_intersected.to_file(shapefile_path + '/grid_with_intersected.shp')
#
# # Step 6: Rasterize the intersected polygons
# # Define the raster properties
# transform = rasterio.transform.from_bounds(xmin, ymin, xmax, ymax, len(cols), len(rows))
#
# # Prepare shapes and values for rasterisation:
# shapes = ((geom, value) for geom, value in zip(grid_with_intersected.geometry, grid_with_intersected['SHETRAN']))
#
# # Rasterize:
# raster = rasterio.features.rasterize(
#     shapes,
#     out_shape=(len(rows), len(cols)),
#     transform=transform,
#     fill=-9999,  # NoData value
#     dtype="int32"
# )
#
# # Convert 0s to -9999s for no data values:
# raster[raster == 0] = -9999
#
# write_ascii(
#     array=raster,
#     ascii_ouput_path=f'{root}/Processed Data/CEH_LCM_2007 {resolution_output}m.asc',
#     xllcorner=xmin,
#     yllcorner=ymin,
#     cellsize=cell_size,
#     NODATA_value=-9999,
#     data_format='%1.0f'
# )